# Projeto

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane.optimize import AdamOptimizer
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AngleEmbedding
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import pickle

In [2]:

# Read the HDF5 file using pandas
data_frame_fcnc = pd.read_hdf('fcnc_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_fcnc.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_fcnc.head()


Number of rows: 90548


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,1.408853,15.150869,339.182312,2.350262,1.396943,0.710451,0.109013,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,test,7.762202e-09,0.001285
1,1,-2.481838,7.208333,247.036240,-2.280740,0.428710,0.205213,0.000000,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,val,7.762202e-09,0.001285
2,0,1.476267,94.220718,238.014694,-1.788097,94.256210,2.418446,1.585315,1.127324,0.431098,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
3,1,0.647480,13.459283,230.971832,-1.032663,1.227122,0.467150,0.164008,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
4,0,2.106436,97.490242,698.399902,-3.059983,36.555862,2.937936,1.799140,1.093004,0.589724,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285


In [3]:
# Read the HDF5 file using pandas
data_frame_bkg = pd.read_hdf('bkg_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_bkg.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_bkg.head()


Number of rows: 1002490


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,-1.988600,52.710262,229.350952,0.728242,36.148926,23.039709,16.949991,14.424411,12.000529,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,train,0.000018,11.9635
1,0,0.528382,61.115589,315.538910,-0.863614,32.592808,22.366640,16.285843,13.938633,11.180016,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,test,0.000018,11.9635
2,0,1.479911,98.012802,251.109573,-3.133624,90.252274,33.646885,30.612156,27.973904,23.729696,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
3,2,0.926899,-0.000007,240.909348,0.835656,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
4,0,0.781194,72.234299,206.020386,-0.320449,48.886372,20.743645,16.572512,13.070706,11.269534,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635


## PCA

### VQE

#### PCA (2 Features)

In [4]:
data_frame_fcnc_pca = data_frame_fcnc.copy()
data_frame_bkg_pca = data_frame_bkg.copy()

# Drop the categorical features except label, weights and gen_split
data_frame_fcnc_pca.drop(['gen_decay_filter', 'gen_filter', 'gen_n_btags', 'gen_sample', 'gen_sample_filter','gen_decay2','gen_decay1'], axis=1, inplace=True)
data_frame_bkg_pca.drop(['gen_decay_filter', 'gen_filter', 'gen_n_btags', 'gen_sample', 'gen_sample_filter','gen_decay2','gen_decay1'], axis=1, inplace=True)

# Drop the features that are not in both dataframes
for feature in data_frame_fcnc_pca.columns.values:
    if feature not in data_frame_bkg_pca.columns.values:
        data_frame_fcnc_pca.drop([feature], axis=1, inplace=True)

for feature in data_frame_bkg_pca.columns.values:
    if feature not in data_frame_fcnc_pca.columns.values:
        data_frame_bkg_pca.drop([feature], axis=1, inplace=True)
        
# Join the dataframes
data = pd.concat([data_frame_fcnc_pca, data_frame_bkg_pca])

# Substitute the labels "signal" and "bkg" by 1 and 0
data = data.replace(['signal'], 1)
data= data.replace(['bkg'], 0)


# train, test and validation sets
train = data.loc[data['gen_split'] == 'train']
test = data.loc[data['gen_split'] == 'test']
val = data.loc[data['gen_split'] == 'val']

In [5]:

def perform_PCA (DataFeatures, pca_n_features, train, data):

    ## Fit PCA to train data & rank components by AUC
    pca = PCA(n_components=len(DataFeatures))
    pca.fit(train[DataFeatures])

    ## Transform the desired dataset to get its principal components
    # Get ranked components by AUC from the train data
    principalComponents = pca.transform(train[DataFeatures])

    # Book will be a dictiorary with the AUC (values) of each component (keys)
    book = {}

    # Get values for AUC computation
    y_true = train['gen_label'].values
    weights = train["gen_xsec"].values

    # Renormalise weights
    weights[y_true == 1] = (weights[y_true == 1] / weights[y_true == 1].sum()) * weights.shape[0] / 2
    weights[y_true == 0] = (weights[y_true == 0] / weights[y_true == 0].sum()) * weights.shape[0] / 2

    for feature_idx in range(principalComponents.shape[1]):
        book[f"Component {feature_idx}"] = roc_auc_score(y_true=y_true, y_score=principalComponents[:, feature_idx], sample_weight=weights)

    # Give me the best features
    book = pd.DataFrame.from_dict(book, orient="index")
    book.columns = ["AUC"]
    book.sort_values(by="AUC", ascending=False, inplace=True)
    book.reset_index(inplace=True)
    book.rename(columns={"index": "Feature"}, inplace=True)

    ## Replace current data by its components ##
    # Get components for the current set we want
    principalComponents = pca.transform(data[DataFeatures])

    # Create a new dataframe with PCA data
    newdf = pd.DataFrame(principalComponents, columns=[f"Component {i}" for i in range(principalComponents.shape[1])])

    # Select the best components given their AUC performance in training data
    newdf = newdf[book["Feature"][0 : pca_n_features]]

    # Add the other relevant features
    newdf["gen_xsec"] = data["gen_xsec"].values
    newdf["gen_label"] = data["gen_label"].values
    newdf["gen_split"] = data["gen_split"].values

    # Finally, replace self.data with newdf
    data = newdf

    # Update DataFeatures
    DataFeatures = pd.Index(list(set(data.columns) - set(["gen_label", "gen_xsec", "gen_split"])))
    
    return data, DataFeatures, book

In [6]:
### perform PCA on the train data
# Normalize the data for angle embeding  (Put the data between -pi and pi)
DataFeatures_vqe = pd.Index(list(set(data.columns) - set(["gen_label", "gen_xsec", "gen_split"])))
pca_n_features = 2
data_vqe, DataFeatures_vqe, book = perform_PCA (DataFeatures_vqe, pca_n_features, train, data)

In [7]:
# Split the features array into train, validation and test sets
data_vqe [DataFeatures_vqe] = (((data_vqe [DataFeatures_vqe] - data_vqe [DataFeatures_vqe].min()) / (data_vqe [DataFeatures_vqe].max() - data_vqe [DataFeatures_vqe].min())) * 2 - 1) * (np.pi)

# divide the new data into train, test and validation sets
train = data_vqe.loc[data_vqe['gen_split'] == 'train']
test = data_vqe.loc[data_vqe['gen_split'] == 'test']
val = data_vqe.loc[data_vqe['gen_split'] == 'val']

# divide the train data into signal and background and get 500 samples of each
train_sgn = train.loc[train['gen_label'] == 1].sample(n=500)
train_bkg = train.loc[train['gen_label'] == 0].sample(n=500)
x_train = pd.concat([train_sgn, train_bkg])
x_train = x_train.sample(frac=1)

# divide the validation data into signal and background and get 500 samples of each
val_sgn = val.loc[val['gen_label'] == 1].sample(n=500)
val_bkg = val.loc[val['gen_label'] == 0].sample(n=500)
x_val = pd.concat([val_sgn, val_bkg])
x_val = x_val.sample(frac=1)

# divide the test data into signal and background and get 500 samples of each
test_sgn = test.loc[test['gen_label'] == 1].sample(n=500)
test_bkg = test.loc[test['gen_label'] == 0].sample(n=500)
x_test = pd.concat([test_sgn, test_bkg])
x_test = x_test.sample(frac=1)

# get an array with the labels for each set
y_train = x_train['gen_label'].values
y_val = x_val['gen_label'].values
y_test = x_test['gen_label'].values

# get an array with the weights for each set
w_train = x_train["gen_xsec"].values
w_val = x_val["gen_xsec"].values
w_test = x_test["gen_xsec"].values

# get an array with the features for each set
x_train = x_train[DataFeatures_vqe].values
x_val = x_val[DataFeatures_vqe].values
x_test = x_test[DataFeatures_vqe].values

In [8]:
# loss function
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

# quantum circuit function
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

# classifier function    
def classifier(n_features, n_layers, weights, x):
        #c = circuit(n_features, n_layers, weights, x)
        dev=qml.device("default.qubit", wires=n_features)
        return qml.QNode(circuit, dev)(n_features, n_layers, weights, x)
    
# cost function    
def cost(n_features, n_layers,weights,X,Y,W):  
        # Compute predictions
        y_scores = [(classifier(n_features, n_layers,weights, x) + 1) / 2 for x in X]

        loss = square_loss(Y, y_scores)
        loss = loss * W
        loss = loss.sum()
        
        return loss
    
# train step function    
def train_step(n_features, n_layers,x_train,y_train, w_train, weights, opt,desc='Training'):
        
        # Only require grad if necessary
        x_train = np.array(x_train, requires_grad=False)
        y_train = np.array(y_train, requires_grad=True)
        w_train = np.array(w_train, requires_grad=False)

        # Compute cost and update weights
        weights, loss = opt.step_and_cost(cost, n_features, n_layers,weights, X=x_train, Y=y_train, W=w_train)

        return loss, weights
    
# validation step function
def validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch,best_weights,desc='Validation'):
    X_val = np.array(x_val, requires_grad=False)
    Y_val = np.array(y_val, requires_grad=False)
    W_val = np.array(w_val, requires_grad=False)

    y_scores = np.array([classifier(n_features, n_layers, weights, x) for x in X_val])
    y_scores = (y_scores + 1) / 2

    W_val[Y_val == 1] = (W_val[Y_val == 1] / W_val[Y_val == 1].sum()) * W_val.shape[0] / 2
    W_val[Y_val == 0] = (W_val[Y_val == 0] / W_val[Y_val == 0].sum()) * W_val.shape[0] / 2

    auc_score = roc_auc_score(y_true=Y_val, y_score=y_scores, sample_weight=W_val)
    loss = cost(n_features, n_layers, weights, X_val, Y_val, W_val)


    if best_score is None or auc_score > best_score:
        best_score = auc_score
        best_score_epoch = epoch_number
        best_weights = weights

    tqdm.write(f"Epoch: {epoch_number}, Validation Loss: {loss:.4f}, AUC Score: {auc_score:.4f}")

    return best_score, best_score_epoch, best_weights
     
# train function
def train(n_features, n_layers, x_train, y_train, learning_rate, weights, max_epochs):
    opt = AdamOptimizer(learning_rate)
    best_score = None
    best_weights = None
    best_score_epoch = None
    epoch_number = 0

    with tqdm(total=max_epochs, desc='Epoch', unit='epoch') as pbar:
        for epoch in range(epoch_number, max_epochs):
            epoch_number = epoch

            loss, nf_nl_weights = train_step(n_features, n_layers, x_train, y_train, w_train, weights, opt, desc='Training')
            
            # Log variable values using tqdm.write
            tqdm.write(f"Epoch: {epoch_number:}, Loss: {loss:.4f}")
            
            
            weights = nf_nl_weights[2:]
            weights = weights[0]

            if epoch_number == max_epochs - 1 or (epoch_number+1)%5==0:
                best_score, best_score_epoch, best_weights = validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch, best_weights,desc='Validation')
                # early stopping
                if epoch_number - best_score_epoch > 30 and epoch_number > 80:
                    tqdm.write(f"Early stopping at epoch {epoch_number}")
                    break

            pbar.update(1)  # Update progress bar
        tqdm.write(f"Best Score: {best_score:.4f}")            
        
    return best_score, best_weights

def test(n_features, n_layers,x_test,y_test,w_test, weights):
        # Remove grad
        X_test = np.array(x_test, requires_grad=False)
        Y_test = np.array(y_test, requires_grad=False)
        W_test = np.array(w_test, requires_grad=False)

        # This will be between -1 and 1, we need to convert to between 0 and 1
        y_scores = np.array([classifier(n_features, n_layers,weights, x) for x in X_test])
        y_scores = (y_scores + 1) / 2

        # Renormalize weights
        W_test[Y_test == 1] = (W_test[Y_test == 1] / W_test[Y_test == 1].sum()) * W_test.shape[0] / 2
        W_test[Y_test == 0] = (W_test[Y_test == 0] / W_test[Y_test == 0].sum()) * W_test.shape[0] / 2

        # Calculate ROC
        auc_score = roc_auc_score(y_true=Y_test, y_score=y_scores, sample_weight=W_test)
        
        return auc_score

In [9]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

lr=0.01

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train,lr, weights, 1000)

Epoch:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 153.9020
Epoch: 1, Loss: 153.8595
Epoch: 2, Loss: 153.8473
Epoch: 3, Loss: 153.8185
Epoch: 4, Loss: 153.7733
Epoch: 4, Validation Loss: 259.3311, AUC Score: 0.6769
Epoch: 5, Loss: 153.7092
Epoch: 6, Loss: 153.6190
Epoch: 7, Loss: 153.4807
Epoch: 8, Loss: 153.2700
Epoch: 9, Loss: 152.9715
Epoch: 9, Validation Loss: 257.3385, AUC Score: 0.6801
Epoch: 10, Loss: 152.5673
Epoch: 11, Loss: 152.0302
Epoch: 12, Loss: 151.3347
Epoch: 13, Loss: 150.4694
Epoch: 14, Loss: 149.4358
Epoch: 14, Validation Loss: 249.8120, AUC Score: 0.6936
Epoch: 15, Loss: 148.2449
Epoch: 16, Loss: 146.9170
Epoch: 17, Loss: 145.4817
Epoch: 18, Loss: 143.9781
Epoch: 19, Loss: 142.4545
Epoch: 19, Validation Loss: 237.0511, AUC Score: 0.7201
Epoch: 20, Loss: 140.9670
Epoch: 21, Loss: 139.5760
Epoch: 22, Loss: 138.3412
Epoch: 23, Loss: 137.3155
Epoch: 24, Loss: 136.5359
Epoch: 24, Validation Loss: 227.8735, AUC Score: 0.7461
Epoch: 25, Loss: 136.0150
Epoch: 26, Loss: 135.7327
Epoch: 27, Loss: 135.6345
Epoc

In [10]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7181271805947544


#### Grid search to find the best hyperparameters

In [11]:
# load the best score, best weights and best learning rate
with open('best_score_pca_vqc.pickle', 'wb') as handle:
    best_score = pickle.load(handle)

with open('best_weights_pca_vqc.pickle', 'wb') as handle:
    best_weights = pickle.load(handle)

with open('best_lr_pca_vqc.pickle', 'wb') as handle:
    best_lr = pickle.load(handle)
    
print (best_score)
print (best_weights)
print (best_lr)


Epoch:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 153.8807
Epoch: 1, Loss: 153.8620
Epoch: 2, Loss: 153.8439
Epoch: 3, Loss: 153.8262
Epoch: 4, Loss: 153.8086
Epoch: 4, Validation Loss: 259.4717, AUC Score: 0.6768
Epoch: 5, Loss: 153.7904
Epoch: 6, Loss: 153.7707
Epoch: 7, Loss: 153.7492
Epoch: 8, Loss: 153.7259
Epoch: 9, Loss: 153.7004
Epoch: 9, Validation Loss: 259.2710, AUC Score: 0.6768
Epoch: 10, Loss: 153.6728
Epoch: 11, Loss: 153.6430
Epoch: 12, Loss: 153.6108
Epoch: 13, Loss: 153.5764
Epoch: 14, Loss: 153.5394
Epoch: 14, Validation Loss: 258.9752, AUC Score: 0.6768
Epoch: 15, Loss: 153.5000
Epoch: 16, Loss: 153.4581
Epoch: 17, Loss: 153.4135
Epoch: 18, Loss: 153.3663
Epoch: 19, Loss: 153.3163
Epoch: 19, Validation Loss: 258.5701, AUC Score: 0.6769
Epoch: 20, Loss: 153.2636
Epoch: 21, Loss: 153.2081
Epoch: 22, Loss: 153.1498
Epoch: 23, Loss: 153.0885
Epoch: 24, Loss: 153.0244
Epoch: 24, Validation Loss: 258.0449, AUC Score: 0.6769
Epoch: 25, Loss: 152.9573
Epoch: 26, Loss: 152.8872
Epoch: 27, Loss: 152.8142
Epoc

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

#### Without PCA (2 Features)

In [ ]:
data_frame_fcnc_pca = data_frame_fcnc.copy()
data_frame_bkg_pca = data_frame_bkg.copy()

# Drop the categorical features except label, weights and gen_split
data_frame_fcnc_pca.drop(['gen_decay_filter', 'gen_filter', 'gen_n_btags', 'gen_sample', 'gen_sample_filter','gen_decay2','gen_decay1'], axis=1, inplace=True)
data_frame_bkg_pca.drop(['gen_decay_filter', 'gen_filter', 'gen_n_btags', 'gen_sample', 'gen_sample_filter','gen_decay2','gen_decay1'], axis=1, inplace=True)

# Drop the features that are not in both dataframes
for feature in data_frame_fcnc_pca.columns.values:
    if feature not in data_frame_bkg_pca.columns.values:
        data_frame_fcnc_pca.drop([feature], axis=1, inplace=True)

for feature in data_frame_bkg_pca.columns.values:
    if feature not in data_frame_fcnc_pca.columns.values:
        data_frame_bkg_pca.drop([feature], axis=1, inplace=True)
        
# Join the dataframes
data_vqe = pd.concat([data_frame_fcnc_pca, data_frame_bkg_pca])

# Substitute the labels "signal" and "bkg" by 1 and 0
data_vqe = data_vqe.replace(['signal'], 1)
data_vqe= data_vqe.replace(['bkg'], 0)

#normalize the data except the categorical features and the weights
DataFeatures_vqe = pd.Index(list(set(data_vqe.columns) - set(["gen_label", "gen_xsec", "gen_split"])))
data_vqe [DataFeatures_vqe] = (((data_vqe [DataFeatures_vqe] - data_vqe [DataFeatures_vqe].min()) / (data_vqe [DataFeatures_vqe].max() - data_vqe [DataFeatures_vqe].min())) * 2 - 1) * (np.pi)

# train, test and validation sets
train = data_vqe.loc[data['gen_split'] == 'train']
test = data_vqe.loc[data['gen_split'] == 'test']
val = data_vqe.loc[data['gen_split'] == 'val']

# divide the train data into signal and background and get 500 samples of each
train_sgn = train.loc[train['gen_label'] == 1].sample(n=500, random_state=42)
train_bkg = train.loc[train['gen_label'] == 0].sample(n=500, random_state=42)
x_train = pd.concat([train_sgn, train_bkg])
x_train = x_train.sample(frac=1, random_state=42)

# divide the validation data into signal and background and get 500 samples of each
val_sgn = val.loc[val['gen_label'] == 1].sample(n=500, random_state=42)
val_bkg = val.loc[val['gen_label'] == 0].sample(n=500, random_state=42)
x_val = pd.concat([val_sgn, val_bkg])
x_val = x_val.sample(frac=1, random_state=42)

# divide the test data into signal and background and get 500 samples of each
test_sgn = test.loc[test['gen_label'] == 1].sample(n=500, random_state=42)
test_bkg = test.loc[test['gen_label'] == 0].sample(n=500, random_state=42)
x_test = pd.concat([test_sgn, test_bkg])
x_test = x_test.sample(frac=1, random_state=42)

# get an array with the labels for each set
y_train = x_train['gen_label'].values
y_val = x_val['gen_label'].values
y_test = x_test['gen_label'].values

# get an array with the weights for each set
w_train = x_train["gen_xsec"].values
w_val = x_val["gen_xsec"].values
w_test = x_test["gen_xsec"].values

# get an array with the features for each set
x_train = x_train[['MissingET_MET', 'Jet1_BTag']].values
x_val = x_val[['MissingET_MET', 'Jet1_BTag']].values
x_test = x_test[['MissingET_MET', 'Jet1_BTag']].values

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train,0.01, weights, 1000)

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

#### Grid search to find the best hyperparameters

In [ ]:
# load the best score, best weights and best learning rate
with open('best_score_sem_pca_vqc.pickle', 'wb') as handle:
    best_score = pickle.load(handle)

with open('best_weights_sem_pca_vqc.pickle', 'wb') as handle:
    best_weights = pickle.load(handle)
 
with open('best_lr_sem_pca_vqc.pickle', 'wb') as handle:
    pickle.dump(best_lr, handle)
    
print (best_score)
print (best_weights)
print (best_lr)

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)